In [1]:
import json
import pandas as pd

# keyboard shortcuts, like sublime/atom
import my_keymap


<IPython.core.display.Javascript object>

In [2]:
# statement of the vote data
remote_url = "https://elections.cdn.sos.ca.gov/sov/2016-general/sov/17-presidential-formatted.xls"
# load the election results from the server
# skip the first row with the party designations, for simplicity, and the footer with totals
df = pd.read_excel(remote_url, skiprows=[1], skipfooter=2)


In [3]:
# reset the index so we can operate easily on it numerically 
df = df.reset_index()
# and give it a better name
df = df.rename(columns={"index": "County"})


In [4]:
# drop the empty and percentage columns, we could retain those or recompute them live
df = df[df.index % 3 == 0]
# drop the now incorrect numerical index
df = df.reset_index(drop=True)
# display first 5
df[0:5]

County  Hillary Clinton  Donald J. Trump  Jill \nStein  Gary Johnson  \
0    Alameda         514842.0          95922.0       17830.0       16906.0   
1     Alpine            334.0            217.0          21.0          25.0   
2     Amador           6004.0          10485.0         227.0         804.0   
3      Butte          41567.0          45144.0        2594.0        4625.0   
4  Calaveras           7944.0          13511.0         424.0        1104.0   

   Gloria Estela\n La Riva  Laurence Kotlikoff  Mike Maturen  Evan McMullin  \
0                   3162.0                27.0          71.0         1291.0   
1                      3.0                 0.0           0.0            1.0   
2                     55.0                 0.0           0.0           59.0   
3                    501.0                 0.0          10.0          340.0   
4                     56.0                 0.0           0.0           22.0   

   Bernard "Bernie" Sanders  Jerry\n White  
0                    4206.0            9.0  
1                       1.0            0.0  
2                     100.0            0.0  
3                     782.0            1.0  
4                      75.0            0.0

In [5]:
# load our manual override json file
override = pd.read_json('override.json' )

override.set_index('County', inplace=True)

In [6]:

# this is the five year, load it
census_df = pd.read_csv('census_data/ACSST5Y2018.S1903_data_with_overlays_2020-01-23T121359.csv', skiprows=[1])

In [7]:
census_df[0:3]

GEO_ID                        NAME  S1903_C01_024M  S1903_C02_024E  \
0  0500000US06001  Alameda County, California            1820            38.9   
1  0500000US06003   Alpine County, California              28            62.1   
2  0500000US06005   Amador County, California             337            57.3   

   S1903_C02_024M  S1903_C03_024E  S1903_C03_024M  S1903_C01_025E  \
0             0.4           95443            1617           94676   
1            13.1           85000           20434              14   
2             2.7           70091            5610            1619   

   S1903_C01_025M  S1903_C02_025E       ...        S1903_C03_022E  \
0            1775            24.7       ...                 70808   
1              15             7.9       ...                     -   
2             241            17.1       ...                 50500   

   S1903_C03_022M  S1903_C01_023E  S1903_C01_023M  S1903_C02_023E  \
0            3253           11203             712             2.9   
1              **               1               3             0.6   
2           16033             300             126             3.2   

   S1903_C02_023M  S1903_C03_023E S1903_C03_023M S1903_C01_024E  \
0             0.2           55664           4156         149052   
1             1.6               -             **            110   
2             1.3           50991           7193           5415   

   S1903_C02_017E  
0            54.3  
1            76.3  
2            69.0  

[3 rows x 242 columns]

In [8]:
census_df["NAME"] = census_df["NAME"].str.replace(' County, California', '')
census_df[0:3]

GEO_ID     NAME  S1903_C01_024M  S1903_C02_024E  S1903_C02_024M  \
0  0500000US06001  Alameda            1820            38.9             0.4   
1  0500000US06003   Alpine              28            62.1            13.1   
2  0500000US06005   Amador             337            57.3             2.7   

   S1903_C03_024E  S1903_C03_024M  S1903_C01_025E  S1903_C01_025M  \
0           95443            1617           94676            1775   
1           85000           20434              14              15   
2           70091            5610            1619             241   

   S1903_C02_025E       ...        S1903_C03_022E  S1903_C03_022M  \
0            24.7       ...                 70808            3253   
1             7.9       ...                     -              **   
2            17.1       ...                 50500           16033   

   S1903_C01_023E  S1903_C01_023M  S1903_C02_023E  S1903_C02_023M  \
0           11203             712             2.9             0.2   
1               1               3             0.6             1.6   
2             300             126             3.2             1.3   

   S1903_C03_023E S1903_C03_023M S1903_C01_024E  S1903_C02_017E  
0           55664           4156         149052            54.3  
1               -             **            110            76.3  
2           50991           7193           5415            69.0  

[3 rows x 242 columns]

In [9]:
# and set the census index to the county name as well
census_df = census_df.set_index("NAME")

In [10]:
census_df[-4:]

GEO_ID  S1903_C01_024M  S1903_C02_024E  S1903_C02_024M  \
NAME                                                                       
Tuolumne  0500000US06109             474            58.0             2.2   
Ventura   0500000US06111            1464            39.1             0.7   
Yolo      0500000US06113             755            41.3             1.4   
Yuba      0500000US06115             469            42.3             2.2   

          S1903_C03_024E  S1903_C03_024M  S1903_C01_025E  S1903_C01_025M  \
NAME                                                                       
Tuolumne           61320            3572            2411             383   
Ventura            86445            1703           43853            1419   
Yolo               82716            3174           10881             667   
Yuba               53664            6447            3597             388   

          S1903_C02_025E  S1903_C02_025M       ...        S1903_C03_022E  \
NAME                                           ...                         
Tuolumne            17.5             2.6       ...                 55278   
Ventura             22.2             0.7       ...                 63626   
Yolo                23.6             1.4       ...                 69712   
Yuba                19.4             2.0       ...                 50837   

          S1903_C03_022M  S1903_C01_023E  S1903_C01_023M  S1903_C02_023E  \
NAME                                                                       
Tuolumne           22156             366             184             2.7   
Ventura             4221            6014             451             3.0   
Yolo                7909            1766             353             3.8   
Yuba                6657             960             257             5.2   

          S1903_C02_023M S1903_C03_023E S1903_C03_023M  S1903_C01_024E  \
NAME                                                                     
Tuolumne             1.3          59677           8506            7984   
Ventura              0.2          53374           3212           77162   
Yolo                 0.8          62000          16602           19046   
Yuba                 1.4          39756          13420            7860   

          S1903_C02_017E  
NAME                      
Tuolumne            68.8  
Ventura             57.2  
Yolo                52.4  
Yuba                51.5  

[4 rows x 241 columns]

In [11]:
# Now, on the orignal voting results dataframe, set the index to county
# so we can easily merge it with the sensus data with the same index
df = df.set_index('County')

In [12]:
# finally we concat on the columns axis
output_df = pd.concat([df, census_df], axis=1, sort=False)
output_df

Hillary Clinton  Donald J. Trump  Jill \nStein  Gary Johnson  \
Alameda                 514842.0          95922.0       17830.0       16906.0   
Alpine                     334.0            217.0          21.0          25.0   
Amador                    6004.0          10485.0         227.0         804.0   
Butte                    41567.0          45144.0        2594.0        4625.0   
Calaveras                 7944.0          13511.0         424.0        1104.0   
Colusa                    2661.0           3551.0          85.0         260.0   
Contra Costa            319287.0         115956.0        8637.0       16123.0   
Del Norte                 3485.0           5134.0         245.0         416.0   
El Dorado                36404.0          49247.0        1425.0        5027.0   
Fresno                  141341.0         124049.0        4300.0        9246.0   
Glenn                     3065.0           5788.0          97.0         375.0   
Humboldt                 33200.0          18373.0        3681.0        2163.0   
Imperial                 32667.0          12704.0         745.0        1221.0   
Inyo                      3155.0           4248.0         214.0         328.0   
Kern                     98689.0         129584.0        2887.0        9359.0   
Kings                    13617.0          18093.0         411.0        1338.0   
Lake                     11496.0          10599.0         930.0        1095.0   
Lassen                    2224.0           7574.0         105.0         473.0   
Los Angeles            2464364.0         769743.0       76465.0       88968.0   
Madera                   17029.0          23357.0         644.0        1558.0   
Marin                   108707.0          21771.0        3109.0        4180.0   
Mariposa                  3122.0           5185.0         189.0         326.0   
Mendocino                22079.0          10888.0        2108.0        1331.0   
Merced                   37317.0          28725.0        1209.0        2410.0   
Modoc                      877.0           2696.0          50.0         122.0   
Mono                      2773.0           2111.0         142.0         222.0   
Monterey                 89088.0          34895.0        2823.0        4547.0   
Napa                     39199.0          17411.0        1262.0        2551.0   
Nevada                   26053.0          23365.0        2189.0        2378.0   
Orange                  609961.0         507148.0       18068.0       46128.0   
Placer                   73509.0          95138.0        2501.0        9245.0   
Plumas                    3459.0           5420.0         186.0         411.0   
Riverside               373695.0         333243.0       11320.0       24089.0   
Sacramento              326023.0         189789.0        9846.0       24412.0   
San Benito               12521.0           7841.0         364.0         815.0   
San Bernardino          340833.0         271240.0       10604.0       21050.0   
San Diego               735476.0         477766.0       22820.0       51663.0   
San Francisco           345084.0          37688.0        9904.0        8883.0   
San Joaquin             121124.0          88936.0        3134.0        8049.0   
San Luis Obispo          67107.0          56164.0        2848.0        6473.0   
San Mateo               237882.0          57929.0        5102.0        9464.0   
Santa Barbara           107142.0          56365.0        3719.0        6748.0   
Santa Clara             511684.0         144826.0       12664.0       25325.0   
Santa Cruz               95249.0          22438.0        4485.0        4325.0   
Shasta                   22301.0          51778.0        1300.0        3382.0   
Sierra                     601.0           1048.0          42.0          98.0   
Siskiyou                  7234.0          11341.0         575.0         875.0   
Solano                  102360.0          51920.0        2771.0        6254.0   
Sonoma                  160435.0          51408.

In [13]:
# two related operations for processing the manual overrides
# first combine first will add anything that's present in the override but does not exist in the output, 
#i.e. the winner field or patching missing data
output_df = output_df.combine_first(override)
# next, in place update for actual manual overriding of data points present in the output already, 
# i.e. the results we downloaded above aren't as up to date as we'd like
output_df.update(override)

In [18]:
# and output the result
output_df.to_json('output.json')
output_df.to_csv('output.csv')

In [15]:
# just some verification - though we're not fully handling NA so these aren't thorough checks (S1903_C03_022E has some missing)

In [16]:
output_df["Hillary Clinton"][output_df["Hillary Clinton"].isna()]

Series([], Name: Hillary Clinton, dtype: float64)

In [17]:
output_df["S1903_C03_022E"][output_df["S1903_C03_022E"].isna()]

Series([], Name: S1903_C03_022E, dtype: object)